In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
#Image Augmentation - Modifying the images of the training set so that the CNN Model does not overlearn the existing images as we will get new images in the test set 
train_datagen = ImageDataGenerator(
        rescale=1./255, #for feature scaling
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#Importing the training dataset
training_set = train_datagen.flow_from_directory(
        'dataset/Train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 87151 images belonging to 2 classes.


In [4]:
# Test Set preprocessing
test_datagen = ImageDataGenerator(rescale=1./255)
#Importing the test dataset
test_set = test_datagen.flow_from_directory(
        'dataset/Test',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary') 

Found 37439 images belonging to 2 classes.


Building the model

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model

In [6]:
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification, so 1 neuron and sigmoid activation


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

_________________________________________________________________


In [37]:
# Train the model
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
)


Epoch 1/25


c:\Users\Yash Phatak\Desktop\Divyanga\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2692/2692 ━━━━━━━━━━━━━━━━━━━━ 410s 152ms/step - accuracy: 0.7754 - loss: 0.4018 - val_accuracy: 0.6729 - val_loss: 1.0634
Epoch 2/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 202s 75ms/step - accuracy: 0.8363 - loss: 0.2750 - val_accuracy: 0.6952 - val_loss: 1.4928
Epoch 3/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 207s 77ms/step - accuracy: 0.8400 - loss: 0.2644 - val_accuracy: 0.6701 - val_loss: 1.1590
Epoch 4/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 208s 77ms/step - accuracy: 0.8415 - loss: 0.2614 - val_accuracy: 0.6951 - val_loss: 1.3324
Epoch 5/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 209s 77ms/step - accuracy: 0.8420 - loss: 0.2581 - val_accuracy: 0.6797 - val_loss: 1.9074
Epoch 6/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 210s 78ms/step - accuracy: 0.8418 - loss: 0.2572 - val_accuracy: 0.6837 - val_loss: 1.4137
Epoch 7/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 203s 75ms/step - accuracy: 0.8408 - loss: 0.2555 - val_accuracy: 0.6865 - val_loss: 2.1304
Epoch 8/25
2692/2692 ━━━━━━━━━━━━━━━━━━━━ 200s 74ms/step - accuracy: 0.8386 - l

In [13]:
# Load the saved model
model.save('model.h5', history)

NameError: name 'history' is not defined

In [8]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_set)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


1170/1170 [==============================] - 56s 48ms/step - loss: 0.6958 - accuracy: 0.4775
Test Loss: 0.695849597454071
Test Accuracy: 0.47752344608306885


Predicting Digits

In [9]:
import numpy as np
from keras.preprocessing import image
from sklearn.metrics import accuracy_score

In [15]:
from  keras.models import load_model
from keras.layers import DepthwiseConv2D

loaded_model = load_model('keras_model.h5')

In [18]:
import tensorflow as tf
import numpy as np
import cv2

# Load the image and preprocess it
image = cv2.imread('dataset/test_camera.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
invert = cv2.bitwise_not(gray)
thresh = cv2.threshold(invert,130,255,cv2.THRESH_BINARY)[1]
cv2.imwrite('inverted_image.jpg',thresh)
test_image = tf.keras.utils.load_img('dataset/test-1.png', target_size=(64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.  # Normalize pixel values to [0, 1]

# Perform prediction
result = loaded_model.predict(test_image)

# Convert the probability to a class label
if result[0][0] > 0.5:  # Threshold for binary classification
    prediction = 'Reversed'
else:
    prediction = 'Normal'

print(prediction)

ValueError: in user code:

    File "c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "c:\Users\Yash Phatak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_t

In [32]:
from keras.preprocessing import image
# Load the image and preprocess it
import cv2
import numpy as np
from keras.preprocessing import image

# Perform thresholding on the image
image = cv2.imread('dataset/test_camera.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

# Save the thresholded image to disk
cv2.imwrite('thresh_image.jpg', thresh)

# Load the thresholded image directly as a numpy array
test_image = cv2.imread('thresh_image.jpg')
test_image = cv2.resize(test_image, (224, 224))
test_image = np.expand_dims(test_image, axis=0)


# Preprocess the image
test_image = preprocess_input(test_image)

# Perform prediction
result = loaded_model.predict(test_image)

# Convert the probability to a class label
if result[0][0] > 0.5:
    print("Reversal")
else:
    print("Normal")


1/1 [==============================] - 0s 55ms/step
Reversal
